In [8]:
from tkinter import *
from functools import partial
from Census21_CACD_Wrapper import APIWrapper
FONT = ("arial",10)

In [9]:
def configure_api():
    '''
    Configures API for use
    '''
    api = APIWrapper()
    return api
  
    
def drop_config_button(text, selection, row, column, command):
    """
    Create a dropdown with a button
    
    Args:
        text: str = The label for dropdown
        selection: iterable object =  The options for the dropdown
        row: int = The row to place the dropdown
        colum: int = The column to place the dropdown
        command: function = The command to execute when the button is clicked
    
    Returns:
        dropdown list as object
    """
    label = Label(text=text, font=FONT)
    label.grid(row=row, column=column)
    
    variable = StringVar(window)
    drop_list = OptionMenu(window, variable, *selection)
    variable.set(text)
    drop_list.grid(row=row+1, column=column)
    
    residence_select = Button(text="Select", command=command)
    residence_select.grid(row=row+2, column=column)
    
    return drop_list


def drop_config(text, selection, row, column):
    """
     Create a dropdown without a button
    
    Args:
        text: str = The label for dropdown
        selection: iterable object =  The options for the dropdown
        row: int = The row to place the dropdown
        colum: int = The column to place the dropdown
    
    Returns:
         dropdown list as object
    """
    label = Label(text=text, font=FONT)
    label.grid(row=row, column=column)
    
    variable = StringVar(window)
    drop_list = OptionMenu(window, variable, *selection)
    variable.set(text)
    drop_list.grid(row=row+1, column=column)
    
    return drop_list


def retrieve_res_input():
    """
    Retrieve the selection of 'residence dropdown list' and returns abbreviation (abbreviation used in API call)
    """
    output = residence_drop.cget('text')
    if output == 'Usual Residence':
        residence_code = 'UR'
    elif output == 'Household':
        residence_code = 'HH'
    else:
        residence_code = 'HRP'
    
    dimension_list = list_dimensions(residence_code)
    create_dimension_selection(dimension_list, residence_code)

    
def execute_api_call(residence_code, dimension_drop_1, dimension_drop_2, regional_drop):
    """
    Query API and save data to csv file in '/data/output/'
    
    Args:
        residence_code: tkinter dropdown object 
        dimension_drop_1: tkinter dropdown object 
        Dimension_drop_2: tkinter dropdown object 
        regional drop: tkinter dropdown object 
    
    Returns:
         Saves Data to .csv
    """
    dim_1 = dimension_drop_1.cget('text')
    dim_2 = dimension_drop_2.cget('text')
    region_dict = api.get_areas_by_pop_type(residence_code)
    region_value = regional_drop.cget('text')
    region = region_dict[region_value]
    data = api.query_api(residence_code, f'{dim_1},{dim_2}', region)
    with open(f'data/output/{residence_code}_{dim_1}_{dim_2}_{region}.csv', 'w') as file:
        file.write(data.to_string(index=False))
    print(data)


def create_dimension_selection(dimension_list, residence_code):
    '''
    Creates the remaining three selection dropdown lists & 'Save Data' button after residence selected
    '''
    global dimension_drop_1
    global dimension_drop_2
    global regional_drop
    # Retrieve list of dimensions 
    dimension_drop_1 = drop_config('Variable 1', dimension_list, 
                             row=2, column=2) 
    dimension_drop_2 = drop_config('Variable 2', dimension_list, 
                             row=2, column=3)
    area_codes = list_areas(residence_code)
    regional_drop = drop_config('Regional Divisions', 
                                            area_codes, row=2, column=4)
    but = Button( text = 'Save Data', command=partial(execute_api_call, residence_code, dimension_drop_1, dimension_drop_2, regional_drop))
    but.grid(column=4, row=4)
    
                                    
def list_dimensions(residence_code):
    '''
    Returns the list of demensions relating to that residence code (HH, HRP, UR) & add option for None
    '''
    dimension_list = list(api.get_dims_by_pop_type(residence_code).values())
    dimension_list.append('None')
    return dimension_list


def list_areas(residence_code):
    '''
    returns a list of areas relating to the selected residence code
    '''
    area_list = list(api.get_areas_by_pop_type(residence_code).keys())
    return area_list


def start_selections():
    '''
    Creates Selection drop down
    '''
    residence_drop = drop_config_button('Residence Selection', ('Usual Residence', 'Household', 'Household Reference Person'), 
                             row=2, column=1, command=retrieve_res_input)
    return residence_drop
 
    
# Configure Window
window = Tk()
window.title('Census API')
window.configure(padx=50, pady=50)

#Configure API
api = configure_api()

dimension_drop_1 = None
dimension_drop_2 = None
regional_drop = None

residence_drop = start_selections()

# Keep window running
window.mainloop()

       ctry            activity_last_week health_in_general    count
0   England                Does not apply    Does not apply        0
1   England                Does not apply  Very good health  8525036
2   England                Does not apply       Good health  1666515
3   England                Does not apply       Fair health   231659
4   England                Does not apply        Bad health    45684
..      ...                           ...               ...      ...
91    Wales  Economically inactive: Other  Very good health    23323
92    Wales  Economically inactive: Other       Good health    27061
93    Wales  Economically inactive: Other       Fair health    17508
94    Wales  Economically inactive: Other        Bad health     5224
95    Wales  Economically inactive: Other   Very bad health     1187

[96 rows x 4 columns]
